In [3]:
import pandas as pd

# Ruta al archivo en tu máquina (usá la r al principio para que tome bien las barras)
ruta_archivo = r'C:\Maestria\Labo 3\sell-in.txt'

# Intentamos leer el archivo con autodetección de separador
df = pd.read_csv(ruta_archivo, sep=None, engine='python')

# Mostramos las primeras filas
print(df.head())



   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  \
0   201701        10234       20524                      0                 2   
1   201701        10032       20524                      0                 1   
2   201701        10217       20524                      0                 1   
3   201701        10125       20524                      0                 1   
4   201701        10012       20524                      0                11   

   cust_request_tn       tn  
0          0.05300  0.05300  
1          0.13628  0.13628  
2          0.03028  0.03028  
3          0.02271  0.02271  
4          1.54452  1.54452  


In [2]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import zipfile

#------------- CONFIG -----------------
RUTA_VENTAS      = Path(r"C:\Maestria\Labo 3\sell-in.txt")    # archivo histórico
RUTA_ZIP         = Path(r"C:\Maestria\Labo 3\Zip-tb.zip")     # contiene tb_productos.txt
NOMBRE_TXT_PROD  = "tb_productos.txt"                         # dentro del zip
WINDOW_MESES     = 3                                          # promedio simple de N últimos meses
RUTA_SALIDA_CSV  = Path(r"C:\Maestria\Labo 3\submission.csv")
#--------------------------------------

# 1) ------------------ Leer datasets ------------------
# Ventas
df = pd.read_csv(RUTA_VENTAS, sep=None, engine="python")   # autodetecta tab, coma, etc.

# Lista de los 780 productos
with zipfile.ZipFile(RUTA_ZIP) as z:
    with z.open(NOMBRE_TXT_PROD) as f:
        productos = pd.read_csv(f, sep=None, engine="python")

# 2) ------- Dejar sólo columnas clave y tipar ---------
df = df[['periodo', 'product_id', 'tn']].copy()
df['periodo'] = df['periodo'].astype(str).str.zfill(6)        # 202504 → string ‘202504’
df['anio']    = df['periodo'].str[:4].astype(int)
df['mes']     = df['periodo'].str[4:6].astype(int)

# 3) ---------- Detectar último periodo  --------------
ultimo_anio, ultimo_mes = df[['anio','mes']].max().tolist()
# mes siguiente
if ultimo_mes == 12:
    prox_anio, prox_mes = ultimo_anio + 1, 1
else:
    prox_anio, prox_mes = ultimo_anio, ultimo_mes + 1
target_periodo_str = f"{prox_anio}{prox_mes:02d}"
print(f"⚙️  Prediciendo para el periodo {target_periodo_str}")

# 4) ---------- Agregar ventas por producto -----------
# Ordenar por fecha para el rolling
df['fecha_idx'] = pd.to_datetime(df['anio'].astype(str) + df['mes'].astype(str) + '01', format="%Y%m%d")
ventas_mensuales = (df.groupby(['product_id', 'fecha_idx'])['tn']
                      .sum()
                      .sort_index()
                      .groupby(level=0))

# 5) ---------- Baseline: media últimos N meses --------
# Calculamos rolling window hacia atrás
predicciones = (ventas_mensuales
                .apply(lambda s: s.tail(WINDOW_MESES).mean())
                .reset_index()
                .groupby('product_id', as_index=False)['tn']
                .last())

# 6) ---------- Garantizar 780 filas -------------------
predicciones = productos[['product_id']].merge(predicciones, on='product_id', how='left')
predicciones['tn'] = predicciones['tn'].fillna(0)   # productos sin ventas recientes ⇒ 0

# 7) ---------- Formatear decimales & guardar ----------
predicciones['tn'] = predicciones['tn'].round(5)
predicciones.to_csv(RUTA_SALIDA_CSV, index=False, float_format="%.5f")

print(f"✅ Archivo guardado en {RUTA_SALIDA_CSV.resolve()}")


⚙️  Prediciendo para el periodo 202001
✅ Archivo guardado en C:\Maestria\Labo 3\submission.csv


In [3]:
import pandas as pd
from pathlib import Path
import zipfile

# ---------- CONFIG ----------
RUTA_VENTAS      = Path(r"C:\Maestria\Labo 3\sell-in.txt")
RUTA_ZIP         = Path(r"C:\Maestria\Labo 3\Zip-tb.zip")
NOMBRE_TXT_PROD  = "tb_productos.txt"
WINDOW_MESES     = 3                                  # media móvil de N meses
RUTA_SALIDA_CSV  = Path(r"C:\Maestria\Labo 3\submission.csv")
# ----------------------------

# 1) Leer ventas
df = pd.read_csv(RUTA_VENTAS, sep=None, engine="python")

# 2) Leer lista de todos los productos (por si hay metadatos útiles)
with zipfile.ZipFile(RUTA_ZIP) as z:
    with z.open(NOMBRE_TXT_PROD) as f:
        df_prod = pd.read_csv(f, sep=None, engine="python")

# 3) Arreglar fechas
df['periodo'] = df['periodo'].astype(str).str.zfill(6)
df['anio']    = df['periodo'].str[:4].astype(int)
df['mes']     = df['periodo'].str[4:6].astype(int)
df['fecha']   = pd.to_datetime(df['anio'].astype(str)+df['mes'].astype(str)+'01', format='%Y%m%d')

# 4) Detectar próximo mes
ult_a, ult_m = df[['anio','mes']].max()
prox_a, prox_m = (ult_a + 1, 1) if ult_m == 12 else (ult_a, ult_m + 1)
print(f"Prediciendo para {prox_a}{prox_m:02d}")

# 5) TOP-779 por tn acumulada
top_ids = (df.groupby('product_id')['tn']
             .sum()
             .sort_values(ascending=False)
             .head(779)
             .index)

# 6) Serie mensual por SKU
df_top = df[df['product_id'].isin(top_ids)]
serie = (df_top.groupby(['product_id', 'fecha'])['tn']
               .sum()
               .sort_index()
               .groupby(level=0))

# 7) Baseline: media últimos N meses
pred = (serie.apply(lambda s: s.tail(WINDOW_MESES).mean())
              .reset_index()
              .groupby('product_id', as_index=False)['tn']
              .last())

# 8) Asegurar exactamente 779 filas (ya lo son) y formateo
pred['tn'] = pred['tn'].fillna(0).round(5)
pred = pred.sort_values('product_id')            # orden opcional
assert len(pred) == 779, "Debe haber 779 productos"

# 9) Guardar CSV
pred.to_csv(RUTA_SALIDA_CSV, index=False, float_format="%.5f")
print(f"CSV generado: {RUTA_SALIDA_CSV} — filas: {len(pred)+1}")


Prediciendo para 202001
CSV generado: C:\Maestria\Labo 3\submission.csv — filas: 780


In [4]:
import pandas as pd
from pathlib import Path
import zipfile

# ---------- CONFIGURA RUTAS ----------
RUTA_VENTAS      = Path(r"C:\Maestria\Labo 3\sell-in.txt")
RUTA_ZIP         = Path(r"C:\Maestria\Labo 3\Zip-tb.zip")
TXT_PRODUCTOS    = "tb_productos.txt"        # nombre dentro del ZIP
WINDOW_MESES     = 3                         # media móvil simple
RUTA_SALIDA_CSV  = Path(r"C:\Maestria\Labo 3\submission_t780.csv")
# --------------------------------------

# 1) Leer ventas
df = pd.read_csv(RUTA_VENTAS, sep=None, engine="python")

# 2) Leer lista de productos (por si la necesitás más adelante)
with zipfile.ZipFile(RUTA_ZIP) as z:
    with z.open(TXT_PRODUCTOS) as f:
        _ = pd.read_csv(f, sep=None, engine="python")   # no la usamos ahora

# 3) Preparar campo fecha
df['periodo'] = df['periodo'].astype(str).str.zfill(6)
df['anio']    = df['periodo'].str[:4].astype(int)
df['mes']     = df['periodo'].str[4:6].astype(int)
df['fecha']   = pd.to_datetime(df['anio'].astype(str)+df['mes'].astype(str)+'01', format='%Y%m%d')

# 4) Detección del próximo mes (solo para información)
ult_a, ult_m = df[['anio','mes']].max()
prox_a, prox_m = (ult_a + 1, 1) if ult_m == 12 else (ult_a, ult_m + 1)
print(f"📆 Prediciendo para: {prox_a}{prox_m:02d}")

# 5) Top-780 por venta total (tn acumulada)
top_ids = (df.groupby('product_id')['tn']
             .sum()
             .sort_values(ascending=False)
             .head(780)         # ←–– AHORA 780
             .index)

# 6) Serie mensual por SKU (solo top)
df_top = df[df['product_id'].isin(top_ids)]
serie  = (df_top.groupby(['product_id', 'fecha'])['tn']
                .sum()
                .sort_index()
                .groupby(level=0))

# 7) Predicción baseline: media de los últimos N meses
pred = (serie.apply(lambda s: s.tail(WINDOW_MESES).mean())
              .reset_index()
              .groupby('product_id', as_index=False)['tn']
              .last())

# 8) Garantizar exactamente 780 filas
pred['tn'] = pred['tn'].fillna(0).round(5)
assert len(pred) == 780, f"Se esperaban 780 filas y hay {len(pred)}"

# 9) Guardar CSV final
pred.to_csv(RUTA_SALIDA_CSV, index=False, float_format="%.5f")
print(f"✅ CSV generado en {RUTA_SALIDA_CSV.resolve()} — filas (sin encabezado): {len(pred)}")


📆 Prediciendo para: 202001
✅ CSV generado en C:\Maestria\Labo 3\submission_t780.csv — filas (sin encabezado): 780


In [5]:
import pandas as pd
import zipfile
from pathlib import Path

# ▸ Rutas ------------------------------------------------------------------
ZIP_PROD   = Path(r"C:\Maestria\Labo 3\Zip-tb.zip")   # el zip que subiste
TXT_INSIDE = "tb_productos.txt"                       # nombre dentro del zip
SUB_FILE   = Path(r"C:\Maestria\Labo 3\submission_t780.csv")  # tu submission
# --------------------------------------------------------------------------

# 1) Leer tb_productos.txt desde el ZIP
with zipfile.ZipFile(ZIP_PROD) as z:
    with z.open(TXT_INSIDE) as f:
        prod = pd.read_csv(f, sep=None, engine="python")   # autodetecta tab, coma…

print(f"🔢 Filas totales en tb_productos:        {len(prod)}")
print(f"🔢 product_id únicos en tb_productos:    {prod['product_id'].nunique()}")
print(f"¿Hay ids duplicados?  {len(prod) != prod['product_id'].nunique()}")

🔢 Filas totales en tb_productos:        1251
🔢 product_id únicos en tb_productos:    1251
¿Hay ids duplicados?  False


Primer codigo que anda

In [10]:
import pandas as pd
from pathlib import Path

# ───────── CONFIG ─────────
RUTA_VENTAS      = Path(r"C:\Maestria\Labo 3\sell-in.txt")
RUTA_LISTA_780   = Path(r"C:\Maestria\Labo 3\780_a_predecir.txt")
WINDOW_MESES     = 3                              # media móvil simple
RUTA_SALIDA_CSV  = Path(r"C:\Maestria\Labo 3\submission_t780.csv")
# ──────────────────────────


# 1) Leer histórico de ventas
df = pd.read_csv(RUTA_VENTAS, sep=None, engine="python")


# 2) Leer lista exacta de 780 product_id  (línea por línea)
product_ids = []
with open(RUTA_LISTA_780, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line or line.lower().startswith('product'):
            continue            # salta encabezado o líneas vacías
        product_ids.append(int(line))

assert len(product_ids) == 780, f"La lista debería tener 780 ids y tiene {len(product_ids)}"
lista = pd.DataFrame({'product_id': product_ids})


# 3) Preparar campo fecha
df['periodo'] = df['periodo'].astype(str).str.zfill(6)
df['anio']    = df['periodo'].str[:4].astype(int)
df['mes']     = df['periodo'].str[4:6].astype(int)
df['fecha']   = pd.to_datetime(df['anio'].astype(str) + df['mes'].astype(str) + '01',
                               format='%Y%m%d')


# 4) Detección del próximo mes (solo info)
ult_a, ult_m = df[['anio','mes']].max()
prox_a, prox_m = (ult_a + 1, 1) if ult_m == 12 else (ult_a, ult_m + 1)
print(f"📆 Prediciendo para: {prox_a}{prox_m:02d}")


# 5) Filtrar sólo los 780 SKU
df_780 = df[df['product_id'].isin(product_ids)]


# 6) Serie mensual por SKU
serie = (df_780
         .groupby(['product_id', 'fecha'])['tn']
         .sum()
         .sort_index()
         .groupby(level=0))


# 7) Predicción baseline: media de los últimos N meses
pred = (serie.apply(lambda s: s.tail(WINDOW_MESES).mean())
              .reset_index()
              .groupby('product_id', as_index=False)['tn']
              .last())


# 8) Unir con la lista y rellenar faltantes con 0
pred = lista.merge(pred, on='product_id', how='left')
pred['tn'] = pred['tn'].fillna(0).round(5)

assert len(pred) == 780, f"Se esperaban 780 filas y hay {len(pred)}"


# 9) Guardar CSV final
pred.to_csv(RUTA_SALIDA_CSV, index=False, float_format="%.5f")
print(f"✅ CSV generado en {RUTA_SALIDA_CSV.resolve()} — filas (sin encabezado): {len(pred)}")



📆 Prediciendo para: 202001
✅ CSV generado en C:\Maestria\Labo 3\submission_t780.csv — filas (sin encabezado): 780


1, 3, 6, 9, 12

In [1]:
import pandas as pd
from pathlib import Path

# ───────── CONFIG ─────────
RUTA_VENTAS     = Path(r"C:\Maestria\Labo 3\sell-in.txt")
RUTA_LISTA_780  = Path(r"C:\Maestria\Labo 3\780_a_predecir.txt")
WINDOWS         = [1, 3, 6, 9, 12]                 # medias móviles a generar
CARPETA_SALIDA  = Path(r"C:\Maestria\Labo 3")      # misma carpeta
# ──────────────────────────


# 1) Leer histórico de ventas
df = pd.read_csv(RUTA_VENTAS, sep=None, engine="python")

# 2) Leer lista exacta de 780 product_id  (línea por línea)
with open(RUTA_LISTA_780, 'r', encoding='utf-8') as f:
    product_ids = [int(line.strip()) for line in f
                   if line.strip() and not line.lower().startswith('product')]

assert len(product_ids) == 780, "La lista debe tener 780 ids"
lista = pd.DataFrame({'product_id': product_ids})

# 3) Preparar campo fecha
df['periodo'] = df['periodo'].astype(str).str.zfill(6)
df['anio']    = df['periodo'].str[:4].astype(int)
df['mes']     = df['periodo'].str[4:6].astype(int)
df['fecha']   = pd.to_datetime(
    df['anio'].astype(str) + df['mes'].astype(str) + '01', format='%Y%m%d'
)

# 4) Filtrar sólo los 780 SKU
df = df[df['product_id'].isin(product_ids)]

# 5) Serie mensual por SKU
serie = (df.groupby(['product_id', 'fecha'])['tn']
           .sum()
           .sort_index()
           .groupby(level=0))

# 6) Generar un CSV por cada ventana móvil
for w in WINDOWS:
    pred = (serie.apply(lambda s: s.tail(w).mean())
                  .reset_index()
                  .groupby('product_id', as_index=False)['tn']
                  .last())

    # Unir con la lista y rellenar faltantes
    pred = lista.merge(pred, on='product_id', how='left')
    pred['tn'] = pred['tn'].fillna(0).round(5)

    assert len(pred) == 780, f"Ventana {w}: deberían ser 780 filas"

    # Guardar
    archivo = CARPETA_SALIDA / f"submission_t780_win{w}.csv"
    pred.to_csv(archivo, index=False, float_format="%.5f")
    print(f"✅ CSV ventana {w} guardado en {archivo.name}")

print("🏁 Listo: se generaron los 5 archivos.")


✅ CSV ventana 1 guardado en submission_t780_win1.csv
✅ CSV ventana 3 guardado en submission_t780_win3.csv
✅ CSV ventana 6 guardado en submission_t780_win6.csv
✅ CSV ventana 9 guardado en submission_t780_win9.csv
✅ CSV ventana 12 guardado en submission_t780_win12.csv
🏁 Listo: se generaron los 5 archivos.


arima

In [3]:
pip install statsmodels

  Using cached statsmodels-0.14.4-cp313-cp313-win_amd64.whl.metadata (9.5 kB)
Using cached statsmodels-0.14.4-cp313-cp313-win_amd64.whl (9.8 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from pathlib import Path
from statsmodels.tsa.arima.model import ARIMA
import warnings

warnings.filterwarnings("ignore")

# ───────── CONFIG ─────────
RUTA_VENTAS     = Path(r"C:\Maestria\Labo 3\sell-in.txt")
RUTA_LISTA_780  = Path(r"C:\Maestria\Labo 3\780_a_predecir.txt")
CARPETA_SALIDA  = Path(r"C:\Maestria\Labo 3")
ARCHIVO_SALIDA  = CARPETA_SALIDA / "submission_t780_arima.csv"

ARIMA_ORDER     = (1, 1, 1)
FALLBACK_MESES  = 12                 # ← ahora la media es sobre 12 meses
# ──────────────────────────


# 1) Leer histórico de ventas
df = pd.read_csv(RUTA_VENTAS, sep=None, engine="python")

# 2) Leer lista exacta de 780 product_id
with open(RUTA_LISTA_780, 'r', encoding='utf-8') as f:
    product_ids = [int(line.strip()) for line in f
                   if line.strip() and not line.lower().startswith('product')]

lista = pd.DataFrame({'product_id': product_ids})

# 3) Preparar campo fecha
df['periodo'] = df['periodo'].astype(str).str.zfill(6)
df['anio']    = df['periodo'].str[:4].astype(int)
df['mes']     = df['periodo'].str[4:6].astype(int)
df['fecha']   = pd.to_datetime(df['anio'].astype(str)+df['mes'].astype(str)+'01',
                               format='%Y%m%d')

# 4) Filtrar sólo los 780 SKU
df = df[df['product_id'].isin(product_ids)]

# 5) Serie mensual por SKU
mensual = (df.groupby(['product_id', 'fecha'])['tn']
             .sum()
             .sort_index())

# 6) Forecast por SKU
predicciones = []
for pid in product_ids:
    serie = mensual.loc[pid] if pid in mensual.index.get_level_values(0) else pd.Series()
    serie = serie.asfreq('MS', fill_value=0)

    if serie.shape[0] < 4 or serie.sum() == 0:
        y_hat = serie.tail(1).values[0] if not serie.empty else 0.0
    else:
        try:
            model = ARIMA(serie, order=ARIMA_ORDER).fit()
            y_hat = model.forecast(1).iloc[0]
        except Exception:
            y_hat = serie.tail(FALLBACK_MESES).mean()

    predicciones.append((pid, max(0, round(float(y_hat), 5))))

pred = pd.DataFrame(predicciones, columns=['product_id', 'tn'])

# 7) Guardar CSV
pred.to_csv(ARCHIVO_SALIDA, index=False, float_format="%.5f")
print(f"✅ CSV ARIMA (respaldo 12 m) guardado en: {ARCHIVO_SALIDA.resolve()} — filas: {len(pred)}")


✅ CSV ARIMA (respaldo 12 m) guardado en: C:\Maestria\Labo 3\submission_t780_arima.csv — filas: 780


ARIMA CON VENTANAS de otros meses

In [5]:
import pandas as pd
from pathlib import Path
from statsmodels.tsa.arima.model import ARIMA
import warnings

warnings.filterwarnings("ignore")

# ───────── CONFIG ─────────
RUTA_VENTAS     = Path(r"C:\Maestria\Labo 3\sell-in.txt")
RUTA_LISTA_780  = Path(r"C:\Maestria\Labo 3\780_a_predecir.txt")
CARPETA_SALIDA  = Path(r"C:\Maestria\Labo 3")
VENTANAS        = [1, 3, 6, 9]           # ← ventanas a probar
ARIMA_ORDER     = (1, 1, 1)
# ──────────────────────────


# 1) Leer histórico de ventas
df = pd.read_csv(RUTA_VENTAS, sep=None, engine="python")

# 2) Leer lista exacta de 780 product_id
with open(RUTA_LISTA_780, 'r', encoding='utf-8') as f:
    product_ids = [int(line.strip()) for line in f
                   if line.strip() and not line.lower().startswith('product')]

lista_ids = pd.DataFrame({'product_id': product_ids})

# 3) Preparar campo fecha
df['periodo'] = df['periodo'].astype(str).str.zfill(6)
df['anio']    = df['periodo'].str[:4].astype(int)
df['mes']     = df['periodo'].str[4:6].astype(int)
df['fecha']   = pd.to_datetime(df['anio'].astype(str)+df['mes'].astype(str)+'01', format='%Y%m%d')

# 4) Filtrar los 780 SKU
df = df[df['product_id'].isin(product_ids)]

# 5) Serie mensual por SKU
mensual = (df.groupby(['product_id', 'fecha'])['tn']
             .sum()
             .sort_index())

# 6) Generar un CSV por cada ventana de respaldo
for ventana in VENTANAS:
    predicciones = []

    for pid in product_ids:
        serie = mensual.loc[pid] if pid in mensual.index.get_level_values(0) else pd.Series()
        serie = serie.asfreq('MS', fill_value=0)

        # fallback si la serie es muy corta
        if serie.shape[0] < 4 or serie.sum() == 0:
            y_hat = serie.tail(1).values[0] if not serie.empty else 0.0
        else:
            try:
                model = ARIMA(serie, order=ARIMA_ORDER).fit()
                y_hat = model.forecast(1).iloc[0]
            except Exception:
                y_hat = serie.tail(ventana).mean()

        predicciones.append((pid, max(0, round(float(y_hat), 5))))

    pred = pd.DataFrame(predicciones, columns=['product_id', 'tn'])
    archivo = CARPETA_SALIDA / f"submission_t780_arima_win{ventana}.csv"
    pred.to_csv(archivo, index=False, float_format="%.5f")
    print(f"✅ CSV ventana {ventana} guardado → {archivo.name}")

print("🏁 Listo: se generaron los 4 archivos con ARIMA + fallback dinámico.")


✅ CSV ventana 1 guardado → submission_t780_arima_win1.csv
✅ CSV ventana 3 guardado → submission_t780_arima_win3.csv
✅ CSV ventana 6 guardado → submission_t780_arima_win6.csv
✅ CSV ventana 9 guardado → submission_t780_arima_win9.csv
🏁 Listo: se generaron los 4 archivos con ARIMA + fallback dinámico.
